Меняем ссылку по заданию и синсет



In [1]:
file_URL = "https://courses.openedu.ru/assets/courseware/v1/deb5c0c36ff1cfe6f530ea6a8de2f0d2/asset-v1:ITMOUniversity+AUTTEXTAN+fall_2021_ITMO+type@asset+block/Task_4_sample_12.csv"
synset_task = "travel.n.01"

Запускаем код

In [2]:
import nltk
import urllib
from itertools import product
from scipy.stats import spearmanr
from nltk.corpus import wordnet as wn

file = urllib.request.urlopen(file_URL)
nltk.download('wordnet', quiet=True)
nltk.download('omw', quiet=True)

triples = [line.decode("utf-8").strip().split(",") for line in file]
triples.pop(0)
score_map = {tuple(triple[:2]): float(triple[2]) for triple in triples}

list_pairs = list(score_map)
wup_list, true_list, path_list, lch_list = [], [], [], []

for w1, w2 in list_pairs:

  try:
    all_w1 = wn.synsets(w1, pos="n")
    all_w2 = wn.synsets(w2, pos="n")

    wup = max([item1.wup_similarity(item2) for item1, item2 in product(all_w1, all_w2)])
    wup_list.append(wup)

    lch = max([item1.lch_similarity(item2) for item1, item2 in product(all_w1, all_w2)])
    lch_list.append(lch)

    path = max([item1.path_similarity(item2) for item1, item2 in product(all_w1, all_w2)])
    path_list.append(path)
    
    true_list.append(score_map[(w1, w2)])

  except Exception as e:
    print(w1, w2, "error:", e)

coef, p = spearmanr(path_list, true_list)
print("path Spearman R: %.3f" % coef)

coef, p = spearmanr(lch_list, true_list)
print("lch  Spearman R: %.3f" % coef)

coef, p = spearmanr(wup_list, true_list)
print("wup  Spearman R: %.3f" % coef)
print()

relative = wn.synset(synset_task)
print("Название синсета:", relative.hyponyms()[0].name())
print("Кол-во синсетов:", len(relative.hyponyms()))

path Spearman R: 0.617
lch  Spearman R: 0.617
wup  Spearman R: 0.639

Название синсета: air_travel.n.01
Кол-во синсетов: 17
